#  Science Image Calibration notebook

This notebook calibrates science images by removing bias, dark and dividing out flats.

Made by: Harlan Shaw <harlan.shaw@ucalgary.ca>

## Required Python packages:
[Astropy](https://www.astropy.org/)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/index.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)

It's recommended you download and install [Anaconda](https://www.anaconda.com/products/individual#Downloads) as this contains a Python environment and Astropy.

You will need to install CCDProc using the Conda install command: `conda install -c conda-forge ccdproc`

This will also install Astropy and Astro-SCRAPPY if not already installed.

## Citations:
With the projects used above, any use of this notebook requires the citations of the Python packages and papers from which they are derived. 
Their citations are located here:

[Astropy](https://www.astropy.org/acknowledging.html)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/citation.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)


## REQUIRED VARIABLES

The next cell has variables that must be set for the notebook to function

In [1]:
PATH_TO_FLAT = "reduced/BNmasterflat.fit"
PATH_TO_DARK = "reduced/combined_dark.fit"
PATH_TO_BIAS = "reduced/combined_bias.fit"
PATH_TO_DATA = "I:\\Work\\Tycho\\2021Aug30AsteroidTestField"


In [3]:
from pathlib import Path
from astropy.nddata import CCDData
from astropy.units import adu, second
from ccdproc import ImageFileCollection, subtract_bias, subtract_dark, flat_correct
import numpy as np

path_to_flat = Path(PATH_TO_FLAT)
path_to_dark = Path(PATH_TO_DARK)
path_to_bias = Path(PATH_TO_BIAS)
path_to_data = Path(PATH_TO_DATA)

data_files = ImageFileCollection(path_to_data, glob_include="*.fit*")
combined_bias = CCDData.read(path_to_bias, unit=adu)
combined_dark = CCDData.read(path_to_dark, unit=adu)
combined_flat = CCDData.read(path_to_flat, unit=adu)
output_directory = path_to_data.joinpath("corrected_b_d")
output_directory.mkdir(parents=True, exist_ok=True)

for file_name in data_files.files:
    file_name_stem = Path(file_name).stem
    full_path = path_to_data.joinpath(file_name)
    new_filename = full_path.with_stem(file_name_stem + "_b_d")
    output_file = output_directory.joinpath(new_filename.name)
    data_ccd = CCDData.read(full_path, unit=adu)
    data_ccd = subtract_bias(data_ccd, combined_bias)
    data_ccd = subtract_dark(ccd=data_ccd, master=combined_dark, exposure_time="EXPTIME", exposure_unit=second, scale=False)
    # data_ccd = flat_correct(ccd=data_ccd, flat=combined_flat)
    data_ccd.meta["HISTORY"] = f"Subtracted master bias {path_to_bias.name}"
    data_ccd.meta["HISTORY"] = f"Subtracted master dark {path_to_dark.name}"
    # data_ccd.meta["HISTORY"] = f"Divided master flat {path_to_flat.name}"
    data_ccd.write(output_file)


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
